In [11]:
import pandas as pd
import os

from pymongo import MongoClient

### Limpa Dataset de tipos de infrações

In [62]:
df_tipos_infracoes_2018 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/tipos-infracoes-2018.xlsx")
df_tipos_infracoes_2018 = df_tipos_infracoes_2018.rename(columns = {'Unnamed: 0': 'TIPO DE INFRACAO'})

df_tipos_infracoes_2018.head(n = 20)

,TIPO DE INFRACAO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,ALCOOLEMIA,912,764,324,356,742,334,759,807,892,567,571,610
1,INABILITADO,623,367,169,271,443,179,473,540,448,404,286,289
2,VEÍCULO NÃO LICENCIADO,1602,808,439,349,927,397,922,917,758,877,700,747
3,DIRIGIR UTILIZANDO TELEFONE CELULAR,709,484,202,366,405,188,420,456,377,412,273,321
4,CINTO DE SEGURANÇA,4372,3431,941,1223,2521,1222,3098,3143,2028,3034,2686,2587
5,CADEIRINHA,138,169,42,53,115,52,116,137,96,132,106,97
6,FAIXA DE PEDESTRE,635,482,239,303,568,268,482,452,446,526,503,399
7,ESTACIONAMENTO,21618,12398,5960,7318,8984,4159,11131,10767,10027,11737,9622,8606
8,AVANÇO DE SEMÁFORO,8475,6488,4718,7690,8721,2995,6467,7214,5728,6497,7878,3780


### Limpa Dataset de acidentes com vítimas fatais

In [63]:
df_acidentes_fatais_2008_2017 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/acidentes/acidentes-fatais-2008-2017.xlsx")
df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop("Fatal", axis = 1)
df_acidentes_fatais_2008_2017['Ano'] = 0


for ano in range(2008, 2018):
    df_acidentes_fatais_2008_2017.loc[df_acidentes_fatais_2008_2017['Ano ' + str(ano)] == 1.0, 'Ano'] = str(ano)
    df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop('Ano ' + str(ano), axis = 1)

df_acidentes_fatais_2008_2017.head()

,Região Administrativa,Nome da Via e Trecho,Descrição do Endereço,Natureza,Data,Dia da Semana,Hora,Faixa Horária (6x6),Nº de Vit Fatais,Nº de Vit Feridas,Veículos Múltiplos,Ano
0,ÁGUAS CLARAS,ALAMEDA DOS EUCALIPTOS,"ALAMEDA DOS EUCALIPTOS, QD 107, PRÓX. AO RESTA...",CHOQ OBJ FIXO,05/03/2015,5-QUI,20:00,18:00 - 23:59,1.0,-,MOTOCICLETA,2015
1,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, PRÓX. À UNIVERSIDADE CATÓLIC...",CAPOTAMENTO,11/05/2015,2-SEG,06:00,06:00 - 11:59,1.0,1,CAMINHONETA,2015
2,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO LOTE 48",COLISÃO,08/04/2011,6-SEX,18:10,18:00 - 23:59,1.0,-,CAMINHÃO x BICICLETA,2011
3,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO POSTO PO...",COLISÃO TRASEIRA,24/12/2009,5-QUI,11:40,06:00 - 11:59,1.0,-,ÔNIBUS x BICICLETA,2009
4,ÁGUAS CLARAS,AV. ARNIQUEIRA,"AV. ARNIQUEIRA, QD. 04, PRÓX. Á CHÁC. 81",ATROP PEDESTRE,17/11/2011,5-QUI,17:00,12:00 - 17:59,1.0,-,CAMINHONETA,2011


### Funções para acesso ao banco

In [68]:
client = MongoClient("mongodb://localhost:27017/")
db = client['dados_de_transito']


def dataframe_to_mongo(df, collection):
    
    records = df.to_dict('records')
    db[collection].insert_many(records)

def mongo_to_dataframe(collection):

    cursor = db[collection].find() 
    df = pd.DataFrame(list(cursor))
    del df['_id']
    
    return df

def clear_database():
    db.drop_collection("tipos_infracoes_2018")

### Refresh Database

In [69]:
clear_database()

### Salvar e recuperar DataFrames

In [70]:
dataframe_to_mongo(df_tipos_infracoes_2018, "tipos_infracoes_2018")
dataframe_to_mongo(df_acidentes_fatais_2008_2017, "acidentes_fatais_2008_2017")

In [72]:
test = mongo_to_dataframe("acidentes_fatais_2008_2017")
test.head(n = 20)

,Região Administrativa,Nome da Via e Trecho,Descrição do Endereço,Natureza,Data,Dia da Semana,Hora,Faixa Horária (6x6),Nº de Vit Fatais,Nº de Vit Feridas,Veículos Múltiplos,Ano
0,ÁGUAS CLARAS,ALAMEDA DOS EUCALIPTOS,"ALAMEDA DOS EUCALIPTOS, QD 107, PRÓX. AO RESTA...",CHOQ OBJ FIXO,05/03/2015,5-QUI,20:00,18:00 - 23:59,1.0,-,MOTOCICLETA,2015
1,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, PRÓX. À UNIVERSIDADE CATÓLIC...",CAPOTAMENTO,11/05/2015,2-SEG,06:00,06:00 - 11:59,1.0,1,CAMINHONETA,2015
2,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO LOTE 48",COLISÃO,08/04/2011,6-SEX,18:10,18:00 - 23:59,1.0,-,CAMINHÃO x BICICLETA,2011
3,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO POSTO PO...",COLISÃO TRASEIRA,24/12/2009,5-QUI,11:40,06:00 - 11:59,1.0,-,ÔNIBUS x BICICLETA,2009
4,ÁGUAS CLARAS,AV. ARNIQUEIRA,"AV. ARNIQUEIRA, QD. 04, PRÓX. Á CHÁC. 81",ATROP PEDESTRE,17/11/2011,5-QUI,17:00,12:00 - 17:59,1.0,-,CAMINHONETA,2011
5,ÁGUAS CLARAS,AV. BRASÍLIA,"AV. BRASÍLIA, ADE, CONJ. 08, LT 07, PRÓX. À LÍ...",COLISÃO LATERAL,20/06/2016,2-SEG,23:00,18:00 - 23:59,1.0,-,BICICLETA x ÔNIBUS,2016
6,ÁGUAS CLARAS,AV. DAS ARAUCÁRIAS,"AV. DAS ARAUCÁRIAS, ALTURA DO SHOPPING DE ÁGU...",ATROP PEDESTRE,01/10/2008,4-QUA,07:30,06:00 - 11:59,1.0,-,CAMINHONETA,2008
7,ÁGUAS CLARAS,AV. DAS ARAUCÁRIAS,"AV. DAS ARAUCÁRIAS, CRUZ. COM A AV. PAU BRASIL",COLISÃO,21/05/2011,7-SAB,01:30,00:00 - 05:59,1.0,-,MOTOCICLETA x CAMINHONETA,2011
8,ÁGUAS CLARAS,AV. DAS ARAUCÁRIAS,"AV. DAS ARAUCÁRIAS, PRÓX. AO SIGMA, CRUZ. COM ...",COLISÃO TRASEIRA,09/01/2016,7-SAB,01:30,00:00 - 05:59,1.0,-,MOTOCICLETA x CAMINHONETE,2016
9,ÁGUAS CLARAS,AV. DAS CASTANHEIRAS,"AV. DAS CASTANHEIRAS, BALÃO PRÓX. Á UNIEURO",QUEDA VEÍCULO,22/11/2009,1-DOM,19:00,18:00 - 23:59,1.0,-,MOTOCICLETA,2009
